In [2]:
import pandas as pd

data = pd.read_csv('../알츠하이머/data/FS_raw.csv')

In [3]:
# 필요없는 컬럼 제거
df = data.copy()
df = df.drop(['ID','PET ligand type', 'Onset age'], axis=1)

In [4]:
df[df.positivity=='BAPL1']['Diagnosis code'].unique()

array([0, 4, 3, 5, 2, 1], dtype=int64)

In [5]:
# object형 데이터 인코딩
df['positivity'] = df['positivity'].astype('category').cat.codes
df.APOE = df.APOE.map(lambda x: 1 if x in ['E3/E4', 'E4/E4'] else 0)

In [6]:
def f(x):
    if x==5:
        return 2
    elif x in [3,4]:
        return 1
    else:
        return 0
    
ndf = df.copy()
ndf['Diagnosis code'] = ndf['Diagnosis code'].map(f)
ndf_corr = pd.DataFrame(ndf.corr().iloc[:,4].sort_values(ascending=False))
ndf_corr = ndf_corr.rename(columns={'Diagnosis code':'corr'})

strong_corr = ndf_corr[(ndf_corr['corr']>0.3)|(ndf_corr['corr']<=-0.3)]

In [7]:
strong_corr

,corr
Diagnosis code,1.000000
GDS,0.810880
CDR-SOB,0.661618
positivity,0.493981
(SUV)PUT-l,0.483001
(SUV)PUT-r,0.472220
(SUV)GCA-l,0.471965
(SUV)GCA-r,0.471702
(SUV)GCP-l,0.467180
(SUV)GCP-r,0.459210


In [8]:
lt = strong_corr.index
nndf = ndf[lt]

X = nndf.drop('Diagnosis code', axis=1)
y = nndf['Diagnosis code']

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

In [9]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)
rfc.fit(train_x, train_y)

RandomForestClassifier(random_state=42)

In [10]:
rfc_pred= rfc.predict(test_x)
rfc_pred_proba = rfc.predict_proba(test_x)

In [11]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
print(accuracy_score(test_y, rfc_pred))
print(f1_score(test_y, rfc_pred, average='micro'))

0.9411764705882353
0.9411764705882353


In [12]:
from xgboost.sklearn import XGBClassifier
xgbc = XGBClassifier(random_state=42)
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [13]:
xgbc_pred= xgbc.predict(test_x)
xgbc_pred_proba = xgbc.predict_proba(test_x)

In [14]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
print(accuracy_score(test_y, xgbc_pred))
print(f1_score(test_y, xgbc_pred, average='micro'))

0.8823529411764706
0.8823529411764706
